In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from stdpipe import plots

In [ ]:
plt.rc('image', origin='lower', cmap='gray')

In [ ]:
files = {
    '[B]': 'Cleaned_B_20240419_0054.fits',
    '[V]': 'Cleaned_V_20240419_0054.fits',
    '[R]': 'Cleaned_R_20240419_0054.fits',
    '[I]': 'cleanded_I_20240419_0054.fits',
    '[Hα]': 'Cleaned_Ha_M57.fits',
    '[OIII]': 'Cleaned_OIII_M57.fits'
}

In [ ]:
# Define the WCS target coordinates
target_coords = SkyCoord('18:53:34', '33:01:06', unit=(u.hourangle, u.deg))

# Define the size of the region in pixels
region_size = {'y_min': -185, 'y_max': 50, 'x_min': -170, 'x_max': 100}

In [ ]:
# Read the FITS data and extract WCS-centered regions
data = {}
wcs_objects = {}
for filter_name, file in files.items():
    with fits.open(file) as hdul:
        image_data = hdul[0].data
        wcs = WCS(hdul[0].header)
        
        # Convert WCS target coordinates to pixel coordinates
        x_center, y_center = wcs.world_to_pixel(target_coords)
        x_center, y_center = int(x_center), int(y_center)
        
        # Extract the region around the center
        y_min = y_center + region_size['y_min']
        y_max = y_center + region_size['y_max']
        x_min = x_center + region_size['x_min']
        x_max = x_center + region_size['x_max']
        
        region_data = image_data[y_min:y_max, x_min:x_max]
        data[filter_name] = region_data
        wcs_objects[filter_name] = wcs

In [ ]:
# Calculate pixel scale (arcsec/pixel) and determine scale bar size
wcs_sample = list(wcs_objects.values())[0]
pixscale = abs(wcs_sample.pixel_scale_matrix[1, 1]) * 3600  # Convert degrees/pixel to arcsec/pixel
region_width_arcsec = (region_size['x_max'] - region_size['x_min']) * pixscale
scale_length_arcsec = region_width_arcsec / 10  # Use ~1/10th of the region width as the scale bar

# Calculate scale length in pixels
scale_length_pixels = scale_length_arcsec / pixscale

# Calculate frame width and height in arcseconds
frame_width_pixels = region_size['x_max'] - region_size['x_min']
frame_height_pixels = region_size['y_max'] - region_size['y_min']
frame_width_arcsec = frame_width_pixels * pixscale
frame_height_arcsec = frame_height_pixels * pixscale

# Calculate scale bar size
scale_length_arcsec = frame_width_arcsec / 10  # Use ~1/10th of the region width as the scale bar
scale_length_pixels = scale_length_arcsec / pixscale


In [ ]:
# Calculate pixel scale (arcsec/pixel) and frame dimensions in arcseconds
wcs_sample = list(wcs_objects.values())[0]
pixscale = 0.473
# Calculate frame width and height in arcseconds
frame_width_pixels = region_size['x_max'] - region_size['x_min']
frame_height_pixels = region_size['y_max'] - region_size['y_min']
frame_width_arcsec = frame_width_pixels * pixscale
frame_height_arcsec = frame_height_pixels * pixscale

# Calculate scale bar size
scale_length_arcsec = frame_width_arcsec / 10  # Use ~1/10th of the region width as the scale bar
scale_length_pixels = scale_length_arcsec / pixscale

# Output the calculated values
print(f"Pixel scale: {pixscale:.3f} arcsec/pixel")
print(f"Frame dimensions: {frame_width_arcsec:.1f}'' x {frame_height_arcsec:.1f}''")
print(f"Scale bar: {scale_length_arcsec:.1f}'' ({scale_length_pixels:.1f} pixels)")

In [ ]:
# Pixel size (arcsec/pixel)
pixel_size = 0.473  # arcsec/pixel

# Frame dimensions in pixels
frame_width_pixels = region_size['x_max'] - region_size['x_min']
frame_height_pixels = region_size['y_max'] - region_size['y_min']

# Calculate frame size in arcseconds
frame_size_x = frame_width_pixels * pixel_size
frame_size_y = frame_height_pixels * pixel_size

# Create the plot
fig, axes = plt.subplots(2, 3, figsize=(10, 6))
filters_to_plot = ['[B]', '[V]', '[R]', '[I]', '[Hα]', '[OIII]']
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

for i, (ax, filter_name, label) in enumerate(zip(axes.flat, filters_to_plot, labels)):
    img_data = data[filter_name]
    wcs = wcs_objects[filter_name]
    
    # Use stdpipe plotting method directly with colorbar disabled
    plots.imshow(img_data, [0.5, 99.7], interpolation='nearest', ax=ax, show_colorbar=False)
    
    # Remove axis ticks
    ax.axis('off')
    
    # Add filter name inside the subplot
    ax.text(0.05, 0.9, filter_name, color='white', fontsize=10, weight='bold', transform=ax.transAxes)
    
    # Add frame label (a), (b), etc., in the top-right corner
    ax.text(0.95, 0.9, label, color='white', fontsize=10, ha='right', transform=ax.transAxes)

# Set the position for the direction indicators (N and E) in the bottom-right corner
first_ax = axes[0, 0]
arrow_start = (0.9, 0.2)  # Normalized position in the bottom-right corner
arrow_length = 0.08  # Length in normalized coordinates

# Orientation: "Up is 177.4 degrees E of N"
north_angle = np.deg2rad(177.4)  # Correct angle for North (down)
east_angle = north_angle + np.pi / 2  # 90° counter-clockwise for East (left)

# North arrow components
north_dx = arrow_length * np.sin(north_angle)
north_dy = arrow_length * np.cos(north_angle)

# East arrow components
east_dx = arrow_length * np.sin(east_angle)
east_dy = arrow_length * np.cos(east_angle)

# Plot arrows for N and E in the bottom-right corner
first_ax.arrow(arrow_start[0], arrow_start[1], north_dx, north_dy, 
               transform=first_ax.transAxes, color='white', 
               head_width=0.015, head_length=0.02, length_includes_head=True)
first_ax.arrow(arrow_start[0], arrow_start[1], east_dx, east_dy, 
               transform=first_ax.transAxes, color='white', 
               head_width=0.015, head_length=0.02, length_includes_head=True)

# Add labels for N and E in the bottom-right corner
first_ax.text(arrow_start[0] + north_dx - 0.02, arrow_start[1] + north_dy - 0.05, 'N', 
              color='white', fontsize=9, transform=first_ax.transAxes)
first_ax.text(arrow_start[0] + east_dx - 0.035, arrow_start[1] + east_dy -0.015, 'E', 
              color='white', fontsize=9, transform=first_ax.transAxes)


# Top-center position for the scale bar
scale_bar_x = 0.5  # Normalized horizontal position in the middle
scale_bar_y = 0.9  # Normalized vertical position near the top
scale_bar_length = 0.15  # Slightly longer scale bar length
scale_bar_width = 1  # Thinner scale bar width

# Create the scale bar in the first subplot (near the top)
first_ax.plot([scale_bar_x - scale_bar_length / 2, scale_bar_x + scale_bar_length / 2], 
              [scale_bar_y, scale_bar_y], transform=first_ax.transAxes, 
              color='white', lw=scale_bar_width)

# Position text above the scale bar
first_ax.text(scale_bar_x, scale_bar_y + 0.02, f'{scale_length_arcsec:.1f}"', 
              color='white', fontsize=8, ha='center', va='bottom', transform=first_ax.transAxes)

# Add frame size in the bottom-right corner of the first subplot
frame_size_text = f"{frame_size_x:.1f}'' x {frame_size_y:.1f}''"
first_ax.text(0.1, 0.05, frame_size_text, color='white', fontsize=10, 
              ha='left', va='bottom', transform=first_ax.transAxes)

# Adjust layout to remove all spacing between subplots
plt.subplots_adjust(wspace=0, hspace=0)

# Save and show the figure
plt.savefig('adjusted_filters_comparison.png', dpi=300, bbox_inches='tight')
plt.show()